In [1]:
from splinter import Browser
from selenium.common.exceptions import InvalidElementStateException
from selenium.common.exceptions import ElementNotInteractableException
from bs4 import BeautifulSoup
import shutil
import time

In [2]:
# Set the zip codes
codes = []
codes.append('27606')   # just hard code one in
code = codes[0]    # set the variable for zip_and_click

In [3]:
# inititial website to visit
url = 'https://weather.com'

In [4]:
def chrome():
    '''
    Finds the chromedriver in the system and creates a Chrome browser
    '''
    executable_path = {'executable_path': shutil.which('chromedriver')}
    browser = Browser('chrome', **executable_path)
    return(browser)

In [5]:
def zip_and_click(code):
    '''
    Enter zip codes into the search bar on weather.com and click the first result.
    No return, just leaves browser at the first data page.
    '''
    import time
    
    filled = False
    clicked = False
    
    inputs = browser.find_by_tag('input') # get the search box reference
    search_box = inputs[0]
    n=0
    while not filled:
        try:
            n=+1
            search_box.fill(code)
            filled = True
        except InvalidElementStateException:
            time.sleep(1)
    print(n)
#     if search_box.fill(code):
#         print('filled search box')
#     else:
#         print('did not fill search box')
    print(browser.find_link_by_partial_href('/weather/today/l').value)
    while not clicked:
        try:
            browser.click_link_by_partial_href('/weather/today/l')
            print('no exception this time')
            clicked = True
        except ElementNotInteractableException:
            print('exception found....waiting...')
            time.sleep(1)
    return(browser)     

In [6]:
browser = chrome()

In [7]:
browser.visit(url)

In [8]:
zip_and_click(code)

1

exception found....waiting...
no exception this time


In [9]:
location = browser.find_by_xpath('//h1').first.value

In [10]:
timestamp = browser.find_by_tag('p').first.value

In [15]:
def scrape_now(browser):
    ''' Get observed weather data from the current weather page '''
    divs = browser.find_by_tag('div').first
    nowcard = divs.find_by_tag('section.today_nowcard-container').value
    nowlist = nowcard.split('\n')
#     nowcard = divs.find_by_tag('section.today_nowcard-container').value
    return(nowlist)

In [16]:
scrape_now(browser)

['RALEIGH, NC',
 'as of 5:22 pm EST',
 '66°',
 'PARTLY CLOUDY',
 'feels like 66°',
 'H -- L 42°',
 'UV Index 0 of 10',
 'One-Two Punch of Arctic Blasts Could Break Records',
 'RIGHT NOW',
 'Wind',
 'SW 4 mph',
 'Humidity',
 '66%',
 'Dew Point',
 '54°',
 'Pressure',
 '30.05 in',
 'Visibility',
 '10.0 mi']

In [11]:
divs = browser.find_by_tag('div').first
nowcard = divs.find_by_tag('section.today_nowcard-container').value
nowlist = nowcard.split('\n')
nowlist

['RALEIGH, NC',
 'as of 5:22 pm EST',
 '66°',
 'PARTLY CLOUDY',
 'feels like 66°',
 'H -- L 42°',
 'UV Index 0 of 10',
 'One-Two Punch of Arctic Blasts Could Break Records',
 'RIGHT NOW',
 'Wind',
 'SW 4 mph',
 'Humidity',
 '66%',
 'Dew Point',
 '54°',
 'Pressure',
 '30.05 in',
 'Visibility',
 '10.0 mi']

In [67]:
browser.find_by_tag('div#daypart-0').click()
# hour, conditino, high/low, temp, percent_precip
snapshot = browser.find_by_tag('div#daypart-0').value
snaplist = snapshot.split('\n')
snaplist
# for i in range(len(snapshot)):
#     print(browser.find_by_tag('div#daypart-0').value[i])

['TODAY', 'CLOUDY', 'HIGH', '70°', '10%']

In [27]:
dp_details = {}
for i in range(4):
    dp_details['browser.find_by_tag(\'span\#dp\f'{i}\'-details-wind\'').value

'SW 11 mph'

In [49]:
hum = browser.find_by_tag('span.wx-detail-value')
print(hum)

In [51]:
for i in range(4):
    print(hum[i].value)

SW 11 mph
58%
3 of 10
6:43 am5:13 pm


In [12]:
browser.find_by_tag('div.today-daypart-content').first.value

'TODAY\nHIGH\n70°\n10%'

In [19]:
details = browser.find_by_id('dp0-details-narrative').value
details

'Cloudy skies. High near 70F. Winds SW at 10 to 15 mph.'

In [65]:
browser.quit()

1

'Today'

got input box


1

did not find it


In [ ]:
### SEARCH FOR THE TABLE THAT HOLDS THE SEARCH RESULTS ###